##### **Test:** 

$$ H = \sum_{j = 2}^{N} \hat{c}_{1}^{\dagger}\hat{c}_{j} + \hat{c}_{j}^{\dagger}\hat{c}_{1} + U \sum_{j = 2}^{N} \hat{n}_{1}\hat{n}_{j}$$  

Applying Jordan-Wigner (JW) transformation:

$$ H = \sum_{j = 2}^{N} \left( \prod_{i = 2}^{j - 1} \hat{\sigma}_{i}^{z}\right) (\hat{\sigma}_{1}^{-} \hat{\sigma}_{j}^{+} + \hat{\sigma}_{j}^{-} \hat{\sigma}_{1}^{+}) + 
 U \sum_{j = 2}^{N} \hat{\sigma}_{1}^{-}\hat{\sigma}_{1}^{+}\hat{\sigma}_{j}^{-}\hat{\sigma}_{j}^{+} $$



In [1]:
using ITensors
using LinearAlgebra

In [2]:
ITensors.op(::OpName"σz",::SiteType"S=1/2") =
 [1 0
  0 -1]

In [3]:
ITensors.op(::OpName"σ+",::SiteType"S=1/2") =
 [0 1
  0 0]

In [4]:
ITensors.op(::OpName"σ-",::SiteType"S=1/2") =
 [0 0
  1 0]

In [5]:
ITensors.op(::OpName"σ-σ+",::SiteType"S=1/2") =
 [0 0
  0 1]

In [6]:
σ_z = [1 0; 0 -1]
σ_plus = [0 1; 0 0]
σ_minus = [0 0; 1 0]
Identity = [1 0; 0 1] 

σ_minus*σ_z == σ_minus && σ_z*σ_plus == σ_plus

true

In [7]:
σ_minus*σ_plus

2×2 Matrix{Int64}:
 0  0
 0  1

In [8]:
#ITensors Code:

#Fermions

function Create_MPO_using_Fermion(N, U = 1.0, sites = []) 

    if length(sites) == 0
        sites = siteinds("Fermion",N)    
    end
    
    os = OpSum() 
    for j=2:N 
        os += 1,"Cdag",1,"C",j 
        # os += -1,"C",1,"Cdag",j 
        os += 1,"Cdag",j,"C",1 
        os += U,"N",1,"N",j
        
    end 
    
    # Convert these terms to an MPO 
    H = MPO(os,sites)
    return H, sites
end

function Calculate_Correlation_ij_Fermions(psi, i, j)

    #Here the conmutation relations are not taken into account. ITensors just do it by itself inside OPSum() and correlation_matrix().
    sites = siteinds(psi)
    Op_1 = op("Cdag", sites[i])
    Op_2 = op("C", sites[j])

    return inner(psi, apply(Op_1, apply(Op_2,psi))) #It does not always results with the right sign.
end

#Spins

function Create_MPO_using_Spins(N, U = 1.0, sites = []) # Make N S=1/2 spin indices 

    if length(sites) == 0
        sites = siteinds("S=1/2",N) 
    end
    
    os = OpSum()
    for j=2:N 

        OP_array = []
        for i=2:j-1
            x = ["σz";i]
            append!(OP_array, x)   
        end
        append!(OP_array, ["σ-";1; "σ+";j]) 

        os += tuple(OP_array...)
        
        OP_array = []
        for i=2:j-1
            x = ["σz";i]
            append!(OP_array, x)    
        end
        append!(OP_array, ["σ-";j; "σ+";1]) 
        
        os += tuple(OP_array...)

        os += U,"σ-",1,"σ+",1,"σ-",j,"σ+",j
    end 
    
    H = MPO(os,sites)
    return H, sites
end

function Build_MPO_Cdag(j, sites)

    os = OpSum()
    OP_array = []
    for i=1:j-1
        x = ["σz";i]
        append!(OP_array, x)   
    end
    append!(OP_array, ["σ-";j]) 
    os += tuple(OP_array...)
    
    return MPO(os,sites)

end

function Build_MPO_C(j, sites)

    os = OpSum()
    OP_array = []
    for i=1:j-1
        x = ["σz";i]
        append!(OP_array, x)   
    end
    append!(OP_array, ["σ+";j]) 
    os += tuple(OP_array...)
    
    return MPO(os,sites)

end

function Calculate_Correlation_ij_Spins(psi, i, j)
    # <Ψ(t)|b†_i*b_j|Ψ(t)>
    
    sites = siteinds(psi)
    OP_1 = Build_MPO_Cdag(i, sites)
    OP_2 = Build_MPO_C(j, sites)
    
    return inner(psi, apply(OP_1, apply(OP_2,psi)))
end

Calculate_Correlation_ij_Spins (generic function with 1 method)

In [9]:
#ED code:

function Enlarge_Matrix_site_j(j, N, matrix)
    # I⊗...⊗I⊗M⊗I...⊗I

    Identity = zeros(size(matrix))
    for i=1:size(matrix)[1]
        Identity[i,i] = 1
    end
    
    M = Identity
    
    if j == 1
        M = matrix
    end    
    
    for i=2:N 
        if i == j
        M = kron(M, matrix)
        else
        M = kron(M, Identity)        
        end
    end

    return M
end

function Enlarge_Matrix_i_Matrix_j(i,j,N, matrix_i, matrix_j)
    # I⊗...⊗I⊗M_i⊗I...⊗I⊗M_j⊗I⊗I...⊗I

    Identity = zeros(size(matrix_i))
    for i=1:size(matrix_i)[1]
        Identity[i,i] = 1
    end

    M = Identity

    
    if j == 1
        M = matrix_j
    end    

    if i == 1
        M = matrix_i
    end   
    
    for k=2:N 
        if k == j
        M = kron(M, matrix_j)
        elseif k == i
        M = kron(M, matrix_i)
        else
        M = kron(M, Identity)        
        end
    end

    return M
end

function Build_H_ED(N, U = 0)

    if N > 14
        return "Error: N > 14" #N = 14 is the biggest that I can run with 32GB of ram.
    end   
    
    H = zeros(2^N, 2^N)
    
    for j=2:N
        Matriz_1j = σ_minus
        Matriz_j1 = σ_plus

        for i=2:N
            if i <= j-1
            Matriz_1j = kron(Matriz_1j, σ_z)
            Matriz_j1 = kron(Matriz_j1, σ_z)
                
            elseif i == j
            Matriz_1j = kron(Matriz_1j, σ_plus)
            Matriz_j1 = kron(Matriz_j1, σ_minus)
                
            else
            Matriz_1j = kron(Matriz_1j, Identity)  
            Matriz_j1 = kron(Matriz_j1, Identity)  
            end
        end 


        Matriz_U = Enlarge_Matrix_i_Matrix_j(1,j, N, σ_minus*σ_plus, σ_minus*σ_plus)
        
        H += Matriz_1j + Matriz_j1 + U*Matriz_U
    end

    return H
end

function Build_Cdag_Matrix(N, j)

    if N > 14
        return "Error: N > 14" #N = 14 is the biggest that I can run with 32GB of ram.
    end   

    if j == 1
        Matrix = σ_minus
    else
        Matrix = σ_z
    end

    for i=2:N 
        if i <= j-1
        Matrix = kron(Matrix, σ_z)
        elseif i == j
        Matrix = kron(Matrix, σ_minus)
        else
        Matrix = kron(Matrix, Identity)  
        end
    end 

    return Matrix

end

function Build_C_Matrix(N, j)

    if N > 14
        return "Error: N > 14" #N = 14 is the biggest that I can run with 32GB of ram.
    end   

    if j == 1
        Matrix = σ_plus
    else
        Matrix = σ_z
    end
    
    for i=2:N 
        if i <= j-1
        Matrix = kron(Matrix, σ_z)
        elseif i == j
        Matrix = kron(Matrix, σ_plus)
        else
        Matrix = kron(Matrix, Identity)  
        end
    end 

    return Matrix

end

Build_C_Matrix (generic function with 1 method)

**Test values**

In [10]:
N = 8
U = 1.5

1.5

**Ground State Energy**

In [11]:
nsweeps = 10
maxdim = [10,20,100,100,200] #maxdim - integer or array of integers specifying the maximum size allowed for the bond dimension or rank of the MPS being optimized
cutoff = [1E-10] #maxdim - integer or array of integers specifying the maximum size allowed for the bond dimension or rank of the MPS being optimized

1-element Vector{Float64}:
 1.0e-10

In [12]:
H, sites = Create_MPO_using_Fermion(N, U) 
psi0 = randomMPS(sites)

energy_ground_state, psi_ground_state = dmrg(H,psi0; nsweeps, maxdim, cutoff)
energy_1_excited_state, psi_1_excited_state = dmrg(H,[psi_ground_state], psi0; nsweeps, maxdim, cutoff)

After sweep 1 energy=-2.3484489869241325  maxlinkdim=4 maxerr=1.11E-15 time=8.644
After sweep 2 energy=-2.6455334787607754  maxlinkdim=10 maxerr=7.95E-11 time=0.030
After sweep 3 energy=-2.6457511454591187  maxlinkdim=6 maxerr=4.80E-11 time=0.006
After sweep 4 energy=-2.645751310888486  maxlinkdim=2 maxerr=4.97E-11 time=0.005
After sweep 5 energy=-2.645751311064563  maxlinkdim=2 maxerr=5.92E-13 time=0.005
After sweep 6 energy=-2.64575131106459  maxlinkdim=2 maxerr=1.79E-16 time=0.005
After sweep 7 energy=-2.6457513110645894  maxlinkdim=2 maxerr=8.88E-17 time=0.017
After sweep 8 energy=-2.645751311064591  maxlinkdim=2 maxerr=5.00E-16 time=0.004
After sweep 9 energy=-2.6457513110645885  maxlinkdim=2 maxerr=2.06E-16 time=0.003
After sweep 10 energy=-2.6457513110645903  maxlinkdim=2 maxerr=6.66E-16 time=0.012
After sweep 1 energy=-1.8745961862112566  maxlinkdim=4 maxerr=4.43E-16 time=0.155
After sweep 2 energy=-1.9998157268911045  maxlinkdim=12 maxerr=5.09E-11 time=0.008
After sweep 3 ener

(-2.0000000000000018, MPS
[1] ((dim=2|id=651|"Link,l=1"), (dim=2|id=408|"Fermion,Site,n=1"))
[2] ((dim=4|id=759|"Link,l=2"), (dim=2|id=478|"Fermion,Site,n=2"), (dim=2|id=651|"Link,l=1"))
[3] ((dim=2|id=393|"Fermion,Site,n=3"), (dim=4|id=248|"Link,l=3"), (dim=4|id=759|"Link,l=2"))
[4] ((dim=2|id=44|"Fermion,Site,n=4"), (dim=4|id=199|"Link,l=4"), (dim=4|id=248|"Link,l=3"))
[5] ((dim=2|id=883|"Fermion,Site,n=5"), (dim=4|id=78|"Link,l=5"), (dim=4|id=199|"Link,l=4"))
[6] ((dim=2|id=978|"Fermion,Site,n=6"), (dim=4|id=776|"Link,l=6"), (dim=4|id=78|"Link,l=5"))
[7] ((dim=2|id=864|"Fermion,Site,n=7"), (dim=2|id=430|"Link,l=7"), (dim=4|id=776|"Link,l=6"))
[8] ((dim=2|id=533|"Fermion,Site,n=8"), (dim=2|id=430|"Link,l=7"))
)

In [13]:
H, sites = Create_MPO_using_Spins(N, U)  
psi0 = randomMPS(sites)

energy_ground_state, psi_ground_state = dmrg(H,psi0; nsweeps, maxdim, cutoff)
energy_1_excited_state, psi_1_excited_state = dmrg(H,[psi_ground_state], psi0; nsweeps, maxdim, cutoff)

After sweep 1 energy=-1.9876247242153746  maxlinkdim=4 maxerr=1.44E-15 time=0.005
After sweep 2 energy=-2.644571212769643  maxlinkdim=12 maxerr=2.83E-11 time=0.007
After sweep 3 energy=-2.645748649647178  maxlinkdim=7 maxerr=8.07E-11 time=0.007
After sweep 4 energy=-2.645751298324078  maxlinkdim=3 maxerr=9.86E-11 time=0.004
After sweep 5 energy=-2.6457513110607014  maxlinkdim=2 maxerr=2.79E-12 time=0.004
After sweep 6 energy=-2.6457513110645903  maxlinkdim=2 maxerr=6.05E-14 time=0.004
After sweep 7 energy=-2.6457513110645885  maxlinkdim=2 maxerr=2.00E-16 time=0.016
After sweep 8 energy=-2.645751311064594  maxlinkdim=2 maxerr=8.70E-17 time=0.004
After sweep 9 energy=-2.6457513110645947  maxlinkdim=2 maxerr=2.49E-16 time=0.003
After sweep 10 energy=-2.645751311064592  maxlinkdim=2 maxerr=7.22E-16 time=0.003
After sweep 1 energy=-1.7797292275509902  maxlinkdim=4 maxerr=1.22E-15 time=0.008
After sweep 2 energy=-1.9994468972255786  maxlinkdim=13 maxerr=2.47E-11 time=0.010
After sweep 3 ener

(-2.0000000000000013, MPS
[1] ((dim=2|id=993|"Link,l=1"), (dim=2|id=844|"S=1/2,Site,n=1"))
[2] ((dim=4|id=181|"Link,l=2"), (dim=2|id=987|"S=1/2,Site,n=2"), (dim=2|id=993|"Link,l=1"))
[3] ((dim=2|id=651|"S=1/2,Site,n=3"), (dim=4|id=363|"Link,l=3"), (dim=4|id=181|"Link,l=2"))
[4] ((dim=2|id=354|"S=1/2,Site,n=4"), (dim=4|id=295|"Link,l=4"), (dim=4|id=363|"Link,l=3"))
[5] ((dim=2|id=849|"S=1/2,Site,n=5"), (dim=4|id=124|"Link,l=5"), (dim=4|id=295|"Link,l=4"))
[6] ((dim=2|id=147|"S=1/2,Site,n=6"), (dim=4|id=133|"Link,l=6"), (dim=4|id=124|"Link,l=5"))
[7] ((dim=2|id=237|"S=1/2,Site,n=7"), (dim=2|id=83|"Link,l=7"), (dim=4|id=133|"Link,l=6"))
[8] ((dim=2|id=779|"S=1/2,Site,n=8"), (dim=2|id=83|"Link,l=7"))
)

In [14]:
H = Build_H_ED(N, U) 
eigen(H).values

256-element Vector{Float64}:
 -2.645751311064591
 -2.000000000000002
 -2.0000000000000018
 -2.000000000000001
 -2.000000000000001
 -2.0
 -1.9999999999999982
 -1.5413812651491121
 -1.5413812651491106
 -1.5413812651491103
 -1.5413812651491103
 -1.5413812651491101
 -1.5413812651491097
  ⋮
  8.339389937671447
  8.339389937671449
  8.33938993767146
  8.33938993767147
  8.999999999999982
  8.999999999999984
  8.999999999999988
  9.000000000000002
  9.000000000000002
  9.000000000000007
  9.720153254455274
 10.5

**Particle Number expected value $\langle N \rangle = \langle C_{j}^{\dagger}C_{j} \rangle$**

In [15]:
j = 6

6

In [16]:
H, sites = Create_MPO_using_Fermion(N, U) 
psi0 = randomMPS(sites)
energy_ground_state, psi_ground_state = dmrg(H,psi0; nsweeps, maxdim, cutoff, outputlevel = 0)

OP = op("N", sites[j])
inner(psi_ground_state, apply(OP, psi_ground_state))

0.07142857142857097

In [17]:
H, sites = Create_MPO_using_Spins(N, U) 
psi0 = randomMPS(sites)
energy_ground_state, psi_ground_state = dmrg(H,psi0; nsweeps, maxdim, cutoff, outputlevel = 0)

OP1 = op("σ-", sites[j])
OP2 = op("σ+", sites[j])
inner(psi_ground_state, apply(OP1, apply(OP2,psi_ground_state)))

0.07142857142857179

In [18]:
H_ED = Build_H_ED(N, U)
values, vectors = eigen(H_ED)
Psi_0 = vectors[:,1]

OP = Enlarge_Matrix_site_j(j, N, σ_minus*σ_plus)
Adjoint(Psi_0)*OP*Psi_0

0.07142857142857134

**Correlations $\langle C_{i}^{\dagger}C_{j} \rangle$**

In [19]:
i,j = 2,5

(2, 5)

In [20]:
H, sites = Create_MPO_using_Fermion(N, U) 
psi0 = randomMPS(sites)
energy_ground_state, psi_ground_state = dmrg(H,psi0; nsweeps, maxdim, cutoff, outputlevel = 0)

Calculate_Correlation_ij_Fermions(psi_ground_state, i, j)

0.07142857142857155

Here we can see one thing very important:

In [21]:
s1 = sites[j]
s2 = sites[j + 1]

op("Cdag", s2)*op("C", s1) == - op("C", s1)*op("Cdag", s2)

false

In [22]:
op("Cdag", s2)*op("C", s1) == op("C", s1)*op("Cdag", s2)

true

**ITensor does not consider conmutation relations when we use op() function. It just does it by itself when we build an MPO using OPsum() or when we calculate correlations using correlation_matrix().**

In [23]:
correlation_matrix(psi_ground_state,"Cdag","C")[i,j]

0.07142857142857147

In [24]:
H, sites = Create_MPO_using_Spins(N, U) 
psi0 = randomMPS(sites)
energy_ground_state, psi_ground_state = dmrg(H,psi0; nsweeps, maxdim, cutoff, outputlevel = 0)

Calculate_Correlation_ij_Spins(psi_ground_state, i, j)

0.07142857142857154

In [25]:
H_ED = Build_H_ED(N, U)
values, vectors = eigen(H_ED)
Psi_0 = vectors[:,1]

OP = Build_Cdag_Matrix(N, i)*Build_C_Matrix(N, j)
Adjoint(Psi_0)*OP*Psi_0

0.07142857142857134

In this case all the options agree, however, during the TEBD we can see that is better use correlation_matrix(). Our function of Calculate_Correlation_ij_Fermions() does not work always.

**Time Evolution of Correlations**

In [38]:
function Time_Evolution_Operator_ED(t, H)
    # U = exp(-itH)

    return U = exp(-1im*t*H)   
end

SWAP = [1 0 0 0; 0 0 1 0; 0 1 0 0; 0 0 0 -1]

function TEBD_Correlation_ij_Fermions(t, Psi_0, U, site_i =1, site_j = 2, maxdim = 40, output = false)
    #|Ψ(t)> = exp(-itH)|Ψ(0)> #Main Function

    N = length(Psi_0)
    cutoff = 1E-8
    dt = 0.01
    s = siteinds(Psi_0)
    
    # Make gates (1,2),(2,3),(3,4),...
    Evolution_Gates = ITensor[]    

    for j in 1:(N - 1)
    
        s1 = s[j]
        s2 = s[j + 1]

        # hj = op("Cdag", s1)*op("C", s2) - op("C", s1)*op("Cdag", s2) + U*op("N", s1)*op("N", s2) #It does not work.
        hj = op("Cdag", s1)*op("C", s2) + op("Cdag", s2)*op("C", s1) + U*op("N", s1)*op("N", s2) #It works well.
        
        push!(Evolution_Gates, exp(-1im * dt/ 2 * hj))

        Sf = op(SWAP, [s1,s2])
        push!(Evolution_Gates, Sf)
    end

    append!(Evolution_Gates, reverse(Evolution_Gates))

    # Correlation_Values = [real(Calculate_Correlation_ij_Fermions(Psi_0, site_i, site_j))] #It does not work. That is because op does not take into account the commutation relations by itself.
    Correlation_Values = [real(correlation_matrix(Psi_0,"Cdag","C")[site_i,site_j])] #It works well.

    Times = [0.0]
    
    Psi_t = Psi_0    
        
    for time in dt:dt:t
        Psi_t = apply(Evolution_Gates, Psi_t; cutoff, maxdim)  #Each that apply is executed an SVD is also executed, maxdim is the maximum of the dimension that we will use from the SVD.
        normalize!(Psi_t)

        append!(Times, time)
        # append!(Correlation_Values, real(Calculate_Correlation_ij_Fermions(Psi_t, site_i, site_j))) #It does not work.
        append!(Correlation_Values, real(correlation_matrix(Psi_t,"Cdag","C")[site_i,site_j])) #It works well.

        if output == true
            println("Correlations for t = "*string(time)*" calculated.")
        end
    end
    
    return Correlation_Values, Times
end

function TEBD_Correlation_ij_Spins(t, Psi_0, U, site_i =1, site_j = 2, maxdim = 40, output = false)
    #|Ψ(t)> = exp(-itH)|Ψ(0)> #Main Function

    N = length(Psi_0)
    cutoff = 1E-8
    dt = 0.01
    s = siteinds(Psi_0)
    
    # Make gates (1,2),(2,3),(3,4),...
    Evolution_Gates = ITensor[]    

    for j in 1:(N - 1)
    
        s1 = s[j]
        s2 = s[j + 1]

        hj = op("σ-", s1)*op("σ+", s2) + op("σ-", s2)*op("σ+", s1) + U*op("σ-σ+", s1)*op("σ-σ+", s2) #The signs of the JW strings are inside the SWAP gate

        push!(Evolution_Gates, exp(-1im * dt/ 2 * hj))

        Sf = op(SWAP, [s1,s2])
        push!(Evolution_Gates, Sf)
    end

    append!(Evolution_Gates, reverse(Evolution_Gates))

    Correlation_Values = [real(Calculate_Correlation_ij_Spins(Psi_0, site_i, site_j))] 

    Times = [0.0]
    
    Psi_t = Psi_0    
        
    for time in dt:dt:t
        Psi_t = apply(Evolution_Gates, Psi_t; cutoff, maxdim)  #Each that apply is executed an SVD is also executed, maxdim is the maximum of the dimension that we will use from the SVD.
        normalize!(Psi_t)

        append!(Times, time)
        append!(Correlation_Values, real(Calculate_Correlation_ij_Spins(Psi_t, site_i, site_j)))

        if output == true
            println("Correlations for t = "*string(time)*" calculated.")
        end
    end
    
    return Correlation_Values, Times
end

TEBD_Correlation_ij_Spins (generic function with 5 methods)

**Time Evolution of correlations**

In [83]:
N = 8
U = 5

i,j = 1,5

tf = 0.2

state = ["0", "0", "1", "1", "1", "1", "0", "0"]

8-element Vector{String}:
 "0"
 "0"
 "1"
 "1"
 "1"
 "1"
 "0"
 "0"

In [84]:
H, sites = Create_MPO_using_Fermion(N, U) 
Initial_MPS = MPS(sites, state)
normalize!(Initial_MPS)

MPS
[1] ((dim=2|id=991|"Fermion,Site,n=1"), (dim=1|id=430|"Link,l=1"))
[2] ((dim=1|id=430|"Link,l=1"), (dim=2|id=223|"Fermion,Site,n=2"), (dim=1|id=383|"Link,l=2"))
[3] ((dim=1|id=383|"Link,l=2"), (dim=2|id=192|"Fermion,Site,n=3"), (dim=1|id=859|"Link,l=3"))
[4] ((dim=1|id=859|"Link,l=3"), (dim=2|id=688|"Fermion,Site,n=4"), (dim=1|id=564|"Link,l=4"))
[5] ((dim=1|id=564|"Link,l=4"), (dim=2|id=143|"Fermion,Site,n=5"), (dim=1|id=77|"Link,l=5"))
[6] ((dim=1|id=77|"Link,l=5"), (dim=2|id=258|"Fermion,Site,n=6"), (dim=1|id=113|"Link,l=6"))
[7] ((dim=1|id=113|"Link,l=6"), (dim=2|id=559|"Fermion,Site,n=7"), (dim=1|id=553|"Link,l=7"))
[8] ((dim=1|id=553|"Link,l=7"), (dim=2|id=736|"Fermion,Site,n=8"))


In [85]:
TEBD_Correlation_ij_Fermions(0.01, Initial_MPS, U, i,j)

([0.0, -0.0007489100200952444], [0.0, 0.01])

In [86]:
Correlation_Values_TEBD_Fermions, Times_TEBD_Fermions = TEBD_Correlation_ij_Fermions(tf, Initial_MPS, U, i,j)

([0.0, -0.0007489100200952444, -0.002977288701614953, -0.006630372718196264, -0.011618086760131999, -0.017817511362878676, -0.02507586153005526, -0.03321424392670063, -0.042032093959364816, -0.05131210810875176  …  -0.07033814307190549, -0.07961534006389234, -0.08842860425706812, -0.09656080401014112, -0.10381158326065931, -0.11000231236058247, -0.11498050835096556, -0.11862351091782437, -0.12084165274093078, -0.12158027054954225], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09  …  0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2])

In [87]:
H, sites = Create_MPO_using_Spins(N, U) 
Initial_MPS = MPS(sites, state)
normalize!(Initial_MPS)

MPS
[1] ((dim=2|id=201|"S=1/2,Site,n=1"), (dim=1|id=296|"Link,l=1"))
[2] ((dim=1|id=296|"Link,l=1"), (dim=2|id=287|"S=1/2,Site,n=2"), (dim=1|id=314|"Link,l=2"))
[3] ((dim=1|id=314|"Link,l=2"), (dim=2|id=604|"S=1/2,Site,n=3"), (dim=1|id=395|"Link,l=3"))
[4] ((dim=1|id=395|"Link,l=3"), (dim=2|id=898|"S=1/2,Site,n=4"), (dim=1|id=245|"Link,l=4"))
[5] ((dim=1|id=245|"Link,l=4"), (dim=2|id=185|"S=1/2,Site,n=5"), (dim=1|id=817|"Link,l=5"))
[6] ((dim=1|id=817|"Link,l=5"), (dim=2|id=132|"S=1/2,Site,n=6"), (dim=1|id=857|"Link,l=6"))
[7] ((dim=1|id=857|"Link,l=6"), (dim=2|id=25|"S=1/2,Site,n=7"), (dim=1|id=228|"Link,l=7"))
[8] ((dim=1|id=228|"Link,l=7"), (dim=2|id=860|"S=1/2,Site,n=8"))


In [88]:
TEBD_Correlation_ij_Spins(0.01, Initial_MPS, U, i,j)

([0.0, -0.0007489100200952438], [0.0, 0.01])

In [89]:
Correlation_Values_TEBD_Spins, Times_TEBD_Spins = TEBD_Correlation_ij_Spins(tf, Initial_MPS, U, i,j)

([0.0, -0.0007489100200952438, -0.0029772887016149578, -0.006630372718196269, -0.011618086760132006, -0.01781751136287866, -0.025075861530055244, -0.03321424392670064, -0.04203209395936482, -0.05131210810875172  …  -0.07033814307190552, -0.07961534006389244, -0.08842860425706811, -0.09656080401014118, -0.10381158326065941, -0.11000231236058264, -0.11498050835096563, -0.11862351091782425, -0.12084165274093071, -0.1215802705495421], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09  …  0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2])

In [90]:
#Converting MPS to vector
index_combiner = combiner(sites, tags = "index")
A = Initial_MPS[1]*Initial_MPS[2]*Initial_MPS[3]*Initial_MPS[4]*Initial_MPS[5]*Initial_MPS[6]*Initial_MPS[7]*Initial_MPS[8]
Initial_Vector = array(index_combiner*A)

#Building the vector
zero = [1;0]
one = [0;1]

Psi_0 = kron(zero, zero, one, one, one, one, zero, zero)
Psi_0 = Psi_0/norm(Psi_0)

println(Initial_Vector == Psi_0)

Correlation_Operator = Build_Cdag_Matrix(N, i)*Build_C_Matrix(N, j)

Times_ED = [0.0]
Correlations_ED = [Adjoint(Psi_0)*Correlation_Operator*Psi_0]

dt = 0.01
U_Evolution = Time_Evolution_Operator_ED(dt, Build_H_ED(N, U))
Psi_t = Psi_0

for k=1:Integer(tf/dt)
    Psi_t = U_Evolution*Psi_t
    Correlation = Adjoint(Psi_t)*Correlation_Operator*Psi_t
    append!(Correlations_ED, real(Correlation))
    append!(Times_ED, dt*k)
end

true


In [91]:
Correlations_ED

21-element Vector{Float64}:
  0.0
 -0.0007484575210219299
 -0.0029753812377327094
 -0.0066258795589769015
 -0.011609971764568764
 -0.01780480688236772
 -0.025057693066615136
 -0.03318986271649882
 -0.04200088041938204
 -0.051273584943384964
 -0.06077944333064036
 -0.07028418498076726
 -0.07955357671468916
 -0.08835919634201951
 -0.09648406230617176
 -0.10372798054852377
 -0.1099124767279013
 -0.11488519218118359
 -0.1185236352614235
 -0.120738195613851
 -0.12147434715381976

In [92]:
Correlation_Values_TEBD_Fermions

21-element Vector{Float64}:
  0.0
 -0.0007489100200952444
 -0.002977288701614953
 -0.006630372718196264
 -0.011618086760131999
 -0.017817511362878676
 -0.02507586153005526
 -0.03321424392670063
 -0.042032093959364816
 -0.05131210810875176
 -0.060825601082901165
 -0.07033814307190549
 -0.07961534006389234
 -0.08842860425706812
 -0.09656080401014112
 -0.10381158326065931
 -0.11000231236058247
 -0.11498050835096556
 -0.11862351091782437
 -0.12084165274093078
 -0.12158027054954225

In [93]:
Correlation_Values_TEBD_Spins

21-element Vector{Float64}:
  0.0
 -0.0007489100200952438
 -0.0029772887016149578
 -0.006630372718196269
 -0.011618086760132006
 -0.01781751136287866
 -0.025075861530055244
 -0.03321424392670064
 -0.04203209395936482
 -0.05131210810875172
 -0.06082560108290114
 -0.07033814307190552
 -0.07961534006389244
 -0.08842860425706811
 -0.09656080401014118
 -0.10381158326065941
 -0.11000231236058264
 -0.11498050835096563
 -0.11862351091782425
 -0.12084165274093071
 -0.1215802705495421

**Conclusion:** Fermion SiteType works well, but is important to check because is easy to made mistakes with the definitions and -. In addition to this, ITensor does not consider conmutation relationships when we need to use the function op(), this is very important in TEBD codes. It is the same with electrn (spinful fermions) sitetype.